## Snowball Relationship Extraction

The new Relex package is a toolkit for performing probabilistic chemical relationship extraction based on semi-supervised online learning. The aim is to train parse expressions probabilistically, removing the need for creating parsers with trial and error.

This overview is based on how to use the code, for a detailed explanation of the algorithm please see the associated paper: https://www.nature.com/articles/sdata2018111


In general, chemical relationships can consist of any number of entities, that is, the elements of a relationship that are linked together to uniquely define it. Here we will focus on a simple Curie Temperature relationship that consists of the following entities:
- A compound
- A specifier
- A synthesis
- A unit

Thus this forms a quaternary relationship. Note the algorithm is generalm and so any number of entities can be specified. You can even make some entities more important than others.

First define a new model, as usual:

In [1]:
from chemdataextractor.relex import Snowball, ChemicalRelationship
from chemdataextractor.model import BaseModel, StringType, ListType, ModelType, Compound
import re
from chemdataextractor.parse import R, I, W, Optional, merge, join, OneOrMore, Any, ZeroOrMore, Start
from chemdataextractor.parse.cem import chemical_name, chemical_label
from chemdataextractor.parse.base import BaseParser
from chemdataextractor.parse.common import lrb, rrb, delim
from chemdataextractor.utils import first
from chemdataextractor.doc import Paragraph, Heading, Sentence
from lxml import etree

class SynthesisRoute(BaseModel):
    specifier = StringType()
    synthesis = StringType()
    MOF = StringType()

Compound.synthesis_routes = ListType(ModelType(SynthesisRoute))

Now define parse elements that describe how to identify the entities in text. Think of these as tagging processes.

In [2]:
# Define a very basic entity tagger
specifier = ((R(u'^[Ss]ynthe((si(s|sed|zed|se|ze))|tic|ses)?$')| R(u'^[Pp]repar(ation|ed|e)?$')| R(u'^[Pp]roduc(tion|e|ed)?$')))('specifier').add_action(join)
specifier.tag = 'specifier'
#units = ((R(u'^[Mm]icrowave$', re.I)+ Optional(R('^[\-‐‑⁃‒–—―−－⁻]$'))+R(u'assisted')))('units').add_action(merge)
#units.tag = 'units'
synthesis = ((R(u'^[SsHh](olv|ydr)othermal(ly)?$')|R(u'^[eEmMsS](lectr|on|echan)ochemical(ly)?$')))('synthesis')
synthesis.tag = 'synthesis'
#synthesis = (R(u'^[SsHh](olv|ydr)othermal(ly)?$')|R(u'^[eEmMsS](lectr|on|echan)ochemical(ly)?$'))
#synthesis.tag = 'synthesis'
chemical_name.tag = 'name'

Note we tag each with a unique identifier that will be used later. Now let the entities in a sentence be any ordering of these (or whatever ordering you feel like). Here we specify that the synthesis and units must coincide, but this does not have to be the case. 

We also define an extremely general parse phrase, this will be used to identify candidate sentences.

In [3]:
# Let the entities be any combination of chemical names, specifier synthesiss and units
entities = ( specifier|chemical_name|synthesis)
# Now create a very generic parse phrase that will match any combination of these entities
synthesis_route_phrase = (entities + OneOrMore(entities | Any()))('synthesis_route')

# List all the entities
synthesis_route_entities = [ specifier, chemical_name,synthesis]

We are now ready to start Snowballing. Lets formalise our ChemicalRelationship passing in the entities, the extraction phrase and a name.

In [4]:
synthesis_route_relationship = ChemicalRelationship(synthesis_route_entities, synthesis_route_phrase, name='synthesis_routes')

Now create a ```Snowball``` object to use on our relationship and point to a path for training.

Here will we use the default parameters:
- TC = 0.95, the minimum Confidence required for a new relationship to be accepted
- Tsim=0.95, The minimum similarity between phrases for them to be clustered together
- learning_rate = 0.5, How quickly the system updates the confidences based on new information
- Prefix_length=1, number of tokens in phrase prefix
- suffix_length = 1, number of tokens in phrase suffix
- prefix_weight = 0.1, the weight of the prefix in determining similarity
- middles_weight = 0.8, the weight of the middles in determining similarity
- suffix_weight  = 0.1, weight of suffix in determining similarity

Note increasing TC and Tsim yields more extraction patterns but stricter rules on new relations
Increasing the learning rate influences how much we trust new information compared to our training
Increasing the prefix/suffix length increases the likelihood of getting overlapping relationships


The training process in online. This means that the user can train the system on as many papers as they like, and it will continue to update the knowledge base. At each paper, the sentences are scanned for any matches to the parse phrase, and if the sentence matches, candidate relationships are formed. There can be many candidate relationships in a single sentence, so the output provides the user will all available candidates. 

The user can specify to accept a relationship by typing in the number (or numbers) of the candidates they wish to accept. I.e. If you want candidate 0 only, type '0' then press enter. If you want 0 and 3 type '0,3' and press enter. If you dont want any, then press any other key. e.g. 'n' or 'no'. 


In [5]:
snowball = Snowball(synthesis_route_relationship)

In [6]:
snowball.train(corpus='tests/data/relex/MOF_synthesis_training_set/')

1/8:   Efficient Mechanochemical Synthesis of MOF-5 for Linear Alkanes Adsorption _ Journal of Chemical & Engineering Data.html



Besides, the Zn2+/BDC ratio had a great impact on the formation of crystalline MOF-5, and the appropriate Zn2+/BDC ratio was 3:1 for mechanochemical synthesis of MOF-5.


Candidate 0 <(Zn2+,name,3,4), (mechanochemical,synthesis,30,31), (synthesis,specifier,31,32)>

Candidate 1 <(Zn2+,name,21,22), (mechanochemical,synthesis,30,31), (synthesis,specifier,31,32)>

...: n



Recently, mechanochemical method have attracted enormous interest worldwide and have been widely used to synthesize some MOFs, such as HKUST-1,(11-13) MOF-14,(14) ZIF-8,(15) and UIO-66.


Candidate 0 <(mechanochemical,synthesis,2,3), (synthesize,specifier,15,16), (HKUST-1,(11-13),name,21,22)>

Candidate 1 <(mechanochemical,synthesis,2,3), (synthesize,specifier,15,16), (ZIF,name,25,26)>

...: 0,1



Prochowicz et al.(17) synthesized MOF-5 using liquid-assisted mechanochemical synthesis and fo

...: n



Solvent-Free Mechanochemical Synthesis of Ultrasmall Nickel Phosphide Nanoparticles and Their Application as a Catalyst for the Hydrogen Evolution Reaction (HER).


Candidate 0 <(Mechanochemical,synthesis,3,4), (Synthesis,specifier,4,5), (Nickel Phosphide Nanoparticles,name,7,10)>

Candidate 1 <(Mechanochemical,synthesis,3,4), (Synthesis,specifier,4,5), (Hydrogen,name,18,19)>

...: n



Liquid-Assisted Mechanochemical Synthesis of Copper Based MOF-505 for the Separation of CO2 over CH4 or N2.


Candidate 0 <(Mechanochemical,synthesis,3,4), (Synthesis,specifier,4,5), (Copper,name,6,7)>

Candidate 1 <(Mechanochemical,synthesis,3,4), (Synthesis,specifier,4,5), (CO2,name,15,16)>

Candidate 2 <(Mechanochemical,synthesis,3,4), (Synthesis,specifier,4,5), (CH4,name,17,18)>

Candidate 3 <(Mechanochemical,synthesis,3,4), (Synthesis,specifier,4,5), (N2,name,19,20)>

...: n



Highly rapid mechanochemical synthesis of a pillar-layer metal-organic framework for efficient CH4/N2 separation

...: n
2/8:   Electrochemical Synthesis of a Microporous Conductive Polymer Based on a Metal–Organic Framework Thin Film - Lu - 2.html



Electrochemical Synthesis of a Microporous Conductive Polymer Based on a Metal–Organic Framework Thin Film - Lu - 2014 - Angewandte Chemie - Wiley Online Library


Candidate 0 <(Electrochemical,synthesis,0,1), (Synthesis,specifier,1,2), (Lu,name,17,18)>

...: n



A new approach to preparing 3D microporous conductive polymer has been demonstrated in the electrochemical synthesis of a porous polyaniline network with the utilization of a MOF thin film supported on a conducting substrate.


Candidate 0 <(electrochemical,synthesis,14,15), (synthesis,specifier,15,16), (polyaniline,name,19,20)>

...: 0
3/8:   Electrochemical synthesis of flower shaped morphology MOFs in an ionic liquid system and their electrocatalytic app.html




Electrochemical synthesis of flower shaped morphology MOFs in an ionic liquid system and their electrocatalytic application to

In [ ]:
0

This training process automatically clusters the sentences you accept and updates the knowlede base. You can check what has been learned by searching in the relex/data folder. 

You can always stop training and start again, or come back to the same training process if you wish, simply load in an existing snowball system using: ```Snowball.load()```

Looking into data/relex/synthesis_routes_patterns.txt, we see what patterns were learned from our training:

 name_1 is a ferromagnetic transition metal exhibiting a high specifier_1 of synthesis_1  units_1 ( with confidence score 1.0

the name_1 nanocrystals show a transition temperature specifier_1 at around synthesis_1  units_1 ( with confidence score 1.0

the specifier_1  synthesis_1  units_1 ) of bulk name_1 suggests with confidence score 1.0

 name_1 is ferromagnetic with a specifier_1 of synthesis_1  units_1 and with confidence score 1.0

, name_1 has recently attracted much attention due to its high specifier_1 ∼ synthesis_1  units_1 ) with confidence score 1.0

 name_1 is probably the most studied half metal because of it high specifier_1 ∼ synthesis_1  units_1 ) with confidence score 1.0

, name_1 has a high spin polarization ( > 95 % )118 and a specifier_1 of synthesis_1  units_1 . with confidence score 1.0

, name_1 ( name_2 ) has received the most attention , due to its high ferroelectric specifier_1 ∼ synthesis_1  units_1 ) with confidence score 1.0




Now let's extract a new relationship from a previously unseen sentence. We will save to a different file so we can see the new clusters afterwards. We hope that the sentence will be similar enough to a previously seen sentence in order for us to extract the new relationship.

In [11]:
snowball.save_file_name = 'synthesis_route_new'
test_sentence = Sentence('In situ electrochemical synthesis of MOF-5 and its application in improving photocatalytic activity of BiOBr')
rels = snowball.extract(test_sentence)
print("Found relationship:", rels)
print(rels[0].confidence)

Found relationship: None


TypeError: 'NoneType' object is not subscriptable

As we can see, we found the right entities. Lets see how confident we are in this relation

In [10]:
print(rels[0].confidence)

TypeError: 'NoneType' object is not subscriptable

Lets look at the new clusters that have updated to reflect the new sentence: in ```curie_test_output_clusters```

Cluster 3 contains 2 phrases

CoS2 is ferromagnetic with a Curie temperature of 116 K and Co9S8 is antiferromagnetic with a Néel temperature above the decomposition temperature.28 The magnetic susceptibility of Ni3S2 was found to be temperature - independent , which is consistent with Pauli paramagnetism.
     
     
BiFeO3 is ferromagnetic with a curie temperature of 1103 K and this is very interesting

The cluster centroid pattern is:  name_1 is ferromagnetic with a specifier_1 of synthesis_1  units_1 and with confidence score 1.0



So our sentence was assigned to Cluster 3 and the new extraction pattern confidence is 1.0.

Of course, this worked because our new sentence was (purposefully) similar to one that already existed in the training set. In order for this to work more gnereally you will need to train on a lot more than 7 examples.